### 基于段落文档(非FAQ)知识的测试
For closed-book QA, each demonstration is formatted as follows to construct few-shot prompts. 
Swapping the question with the evidence (longer distance between questions and answers) is found to consistently yield lower results across all datasets.

#### Evidence: ...
#### Question: ...
#### Answer: ...

In [14]:
import boto3
import re
import json
from typing import List

sm_client = boto3.client("sagemaker-runtime")

# llm_endpoint = 'bloomz-7b1-mt-2023-04-19-09-41-24-189-endpoint'
# llm_model_name = 'bloomz-7b'
llm_endpoint = 'chatglm-2023-04-27-06-17-07-867-endpoint'
llm_model_name = 'chatglm-7b'

In [16]:
def enforce_stop_tokens(text: str, stop: List[str]) -> str:
    """Cut off the text as soon as any stop words occur."""
    if stop is None:
        return text
    
    return re.split("|".join(stop), text)[0]

def Generate(smr_client, llm_endpoint, prompt, llm_name, stop=None, history=[]):
    answer = None
    if llm_name == "chatglm-7b":
        # logger.info("call chatglm...")
        parameters = {
        "max_length": 2048,
        "temperature": 0.01,
        "num_beams": 1, # >1可能会报错，"probability tensor contains either `inf`, `nan` or element < 0"； 即使remove_invalid_values=True也不能解决
        "do_sample": False,
        "top_p": 0.7,
        "logits_processor" : None,
        # "remove_invalid_values" : True
        }
        response_model = smr_client.invoke_endpoint(
            EndpointName=llm_endpoint,
            Body=json.dumps(
            {
                "inputs": prompt,
                "parameters": parameters,
                "history" : history
            }
            ),
            ContentType="application/json",
        )

        json_ret = json.loads(response_model['Body'].read().decode('utf8'))

        answer = json_ret['outputs']
    elif llm_name == "bloomz-7b":
        # logger.info("call bloomz...")
        parameters = {
            # "early_stopping": True,
            "length_penalty": 1.0,
            "max_new_tokens": 200,
            "temperature": 0,
            "min_length": 20,
            "no_repeat_ngram_size": 200,
            # "eos_token_id": ['\n']
        }

        response_model = smr_client.invoke_endpoint(
            EndpointName=llm_endpoint,
            Body=json.dumps(
                {
                    "inputs": prompt,
                    "parameters": parameters
                }
            ),
            ContentType="application/json",
        )
        
        json_ret = json.loads(response_model['Body'].read().decode('utf8'))
        answer = json_ret['outputs'][len(prompt):]

    return enforce_stop_tokens(answer, stop)

### 可以用以下代码调节三个变量, 去实验不同prompt模版和问题以及知识组合的效果
- AWS_Knowledge_QA_Prompt
- post_text
- fewshots_str 

In [44]:
A_Role="customer"
B_Role="MideaBot"
STOP=[f"\n{A_Role}", f"\n{B_Role}"]

post_text = "why does dishwasher not run"
fewshots_str = """Question: Dishwasher does not run or stops during a cycle\nAnswer: Below are some possible causes and the related solution:\n\nCause: There is no power to the dishwasher.\nSolution: Please replace fuse or reset circuit breaker.\n\nCause: The door is not closed tightly and latched.\nSolution: Close and latch the door.\n\nCause: Most cycles pause for 5 minutes near the start of a cycle to let the dishes soak.\nSolution: This is normal, and reduces the energy required to clean the dishes.\n\nCause: Certain cycles repeatedly pause for 15 to 65 seconds during the main wash.\nSolution: This is normal.\n\nCause: The Delay feature is selected.\nSolution: To reset, see “Options” in the Controls section. \n\n\n\nQuestion: Dishwasher will not fill\nAnswer: Below are some possible causes and the related solution:\n\nCause: The water shutoff valve is not turned on.\nSolution: Turn on the water valve.\n\nCause: The door is not closed tightly and latched.\nSolution: Close and latch the door.\n\nCause: Water pressure is too low. Verify a minimum of\nSolution: 20 psi water pressure is maintained during the filling operation.\n\n\n\n\nQuestion: Dishwasher seems to run too long\nAnswer: Below are some possible causes and the related solution:\n\nCause: The water supplied to the dishwasher is not hot enough.\nSolution: Run the water in the adjacent sink until it gets hot before turning on the dishwasher.\n\nCause: The dishwasher runs longer while heating water, and a delay automatically occurs in some wash and rinse cycles until the water reaches the proper temperature.\nSolution: same to previous.\n\nCause: The Sanitize option is selected.\nSolution: The cycle time increases to heat the water to the correct temperature for sanitizing.\n\nCause: The cycle time will vary depending on soil level. When heavy soil is detected, the Normal cycle will automatically increase the cycle time.\nSolution: Dishwasher is connected to the Cold water supply. Verify the dishwasher is connected correctly\n\n"""


AWS_Knowledge_QA_Prompt = """{B} is customer service chatbot of Midea, {B} can answer customers' questions according to the following document segments ```{fewshots}```, and {B} answer ```not find any message``` if not find related message in ```{fewshots}```\n\n{A}: {question}\n{B}: """
final_prompt = AWS_Knowledge_QA_Prompt.format(fewshots=fewshots_str, question=post_text, A=A_Role, B=B_Role)

answer = Generate(sm_client, llm_endpoint, prompt=final_prompt, llm_name=llm_model_name, stop=STOP)
print(answer)

I\'m sorry, but I\'m not able to provide a specific answer to your question as I don\'t have enough information about your problem. Could you please provide more details about the problem, such as the model number of your dishwasher, the error message or any other information that you\'re seeing in the dishwasher?


In [19]:
A_Role="用户"
B_Role="AWSBot"
STOP=[f"\n{A_Role}", f"\n{B_Role}"]

post_text = "什么是AWS Clean Rooms"
fewshots_str = """AWS Clean Rooms帮助您和您的合作伙伴分析和协作处理您的集体数据集，以获得新的见解，而无需相互透露基础数据。您可以使用AWSClean Rooms它在几分钟内创建自己的洁净室，只需几个步骤即可开始分析您的集体数据集。您可以选择要与之协作的合作伙伴，选择他们的数据集，并为参与者配置限制。
借助AWS Clean Rooms，您可以与数千家已经在使用的公司合作AWS。协作不需要将数据移出AWS或加载到其他平台。运行查询时，从其原始位置AWSClean Rooms读取数据并应用内置分析规则来帮助您保持对数据的控制。
AWSClean Rooms提供您可以配置的内置数据访问控制和审计支持控件。这些控件包括：
1.分析规则，限制SQL查询和提供输出限制的机制
2.加密计算，即使Clean Rooms在处理查询时也能保持数据加密，以遵守严格的数据处理政策
3.查询日志，以查看查询并帮助支持审计
"""

AWS_Knowledge_QA_Prompt = """{B} 是AWS的智能客服, {B} 能参考以下文档回答客户的问题 ```{fewshots}```\n\n{A}: {question}\n{B}: """
final_prompt = AWS_Knowledge_QA_Prompt.format(fewshots=fewshots_str, question=post_text, A=A_Role, B=B_Role)

answer = Generate(sm_client, llm_endpoint, prompt=final_prompt, llm_name=llm_model_name, stop=STOP)
print(answer)

AWS Clean Rooms 是 AWS 提供的用于数据集中进行协作和分析的工具，可以帮助用户和合作伙伴在数据集中执行 SQL 查询和分析，而无需将数据导出或移动到其他平台。AWS Clean Rooms 提供了一个洁净室的概念，使用户可以在原始数据位置读取数据并应用内置的分析规则，从而遵守严格的数据处理政策。它还提供了可配置的数据访问控制和审计支持控件，帮助用户确保数据的安全性和合规性。使用 AWS Clean Rooms，用户可以轻松创建自己的数据洁净室，并与数千家已经在使用的公司合作，快速分析数据集并提取新的见解。
